In [22]:
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
import os

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

os.environ["CUDA_VISIBLE_DEVICES"] = "6"

In [ ]:
"""
Refs:
https://www.tensorflow.org/tutorials/structured_data/feature_columns
https://www.bookstack.cn/read/eat_tensorflow2_in_30_days-zh/5-2,%E7%89%B9%E5%BE%81%E5%88%97feature_column.md
https://medium.com/ml-book/demonstration-of-tensorflow-feature-columns-tf-feature-column-3bfcca4ca5c4
"""

In [4]:
data_path='/data00/home/pengsong/Data/heart.csv'
dataframe = pd.read_csv(data_path)
dataframe.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [7]:
train,test=train_test_split(dataframe,test_size=0.2)
train,val=train_test_split(train,test_size=0.2)
print(len(train),'train example')
print(len(val),'validation examples')
print(len(test),'test examples')

193 train example
49 validation examples
61 test examples


In [14]:
def df_to_dataset(dataframe,shuffle=True,batch_size=32):
    dataframe=dataframe.copy()
    labels=dataframe.pop('target')
    ds=tf.data.Dataset.from_tensor_slices((dict(dataframe),labels))
    if shuffle:
        ds=ds.shuffle(buffer_size=len(dataframe))
    ds=ds.batch(batch_size)
    return ds

In [15]:
batch_size=5
train_ds=df_to_dataset(train,batch_size=batch_size)
val_ds=df_to_dataset(val,shuffle=False,batch_size=batch_size)
test_ds=df_to_dataset(test,shuffle=False,batch_size=batch_size)

In [16]:
for feature_batch,label_batch in train_ds.take(1):
    print('Every feature:',list(feature_batch.keys()))
    print('A batch of ages:',feature_batch['age'])
    print('A batch of targets:',label_batch)

Every feature: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
A batch of ages: tf.Tensor([51 50 61 58 57], shape=(5,), dtype=int64)
A batch of targets: tf.Tensor([0 0 0 1 0], shape=(5,), dtype=int64)


In [25]:
example_batch=next(iter(train_ds))[0]
print(example_batch)
# print('------------------')
# print(example_batch[0])

{'age': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([60, 34, 65, 64, 44])>, 'sex': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([1, 1, 0, 0, 1])>, 'cp': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([3, 1, 3, 3, 3])>, 'trestbps': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([140, 118, 155, 140, 140])>, 'chol': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([185, 182, 269, 313, 235])>, 'fbs': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([0, 0, 0, 0, 0])>, 'restecg': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([2, 2, 0, 0, 2])>, 'thalach': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([155, 174, 148, 133, 180])>, 'exang': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([0, 0, 0, 0, 0])>, 'oldpeak': <tf.Tensor: shape=(5,), dtype=float64, numpy=array([3. , 0. , 0.8, 0.2, 0. ])>, 'slope': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([2, 1, 1, 1, 1])>, 'ca': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([0, 0, 0, 0, 0])>, 'thal': <tf.Tensor: shape=(5,), 

In [35]:
def demo(feature_column):
    feature_layer=layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy())
    return feature_layer(example_batch).numpy()

In [26]:
age=feature_column.numeric_column('age')
demo(age)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[60.]
 [34.]
 [65.]
 [64.]
 [44.]]


In [27]:
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
demo(age_buckets)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [30]:
thal=feature_column.categorical_column_with_vocabulary_list('thal',['fixed','normal','reversible'])
thal_one_hot=feature_column.indicator_column(thal)
demo(thal_one_hot)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]


In [31]:
thal_embedding=feature_column.embedding_column(thal,dimension=8)
demo(thal_embedding)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[ 0.18744227  0.1638572   0.05491212  0.21512526  0.01564628  0.44608098
  -0.060509   -0.6131389 ]
 [ 0.18744227  0.1638572   0.05491212  0.21512526  0.01564628  0.44608098
  -0.060509   -0.6131389 ]
 [ 0.18744227  0.1638572   0.05491212  0.21512526  0.01564628  0.44608098
  -0.060509   -0.6131389 ]
 [-0.06446493 -0.5346966   0.2014269  -0.04494372  0.6633062  -0.31392482
   0.18862908  0.02912235]
 [ 0.18744227  0.1638572   0.05491212  0.21512526  0.01564628  0.44608098
  -0.060509   -0.6131389 ]]


In [38]:
crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=10)
a=demo(feature_column.indicator_column(crossed_feature))
print(len(a[0]))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]
10


In [ ]:
# complete code
feature_columns=[]
#数值列
for header in ['age','trestbps','chol','thalach','oldpeak','slope','ca']:
    feature_columns.append(feature_column.numeric_column(header))
#分桶列
age_buckets=feature_column.bucketized_column(age,boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
feature_columns.append(age_buckets)
#分类列
thal = feature_column.categorical_column_with_vocabulary_list('thal', ['fixed', 'normal', 'reversible'])
thal_one_hot = feature_column.indicator_column(thal)
feature_columns.append(thal_one_hot)
#嵌入列
thal_embedding = feature_column.embedding_column(thal, dimension=8)
feature_columns.append(thal_embedding)
#组合列
crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
crossed_feature = feature_column.indicator_column(crossed_feature)
feature_columns.append(crossed_feature)


In [ ]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [ ]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [ ]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'],
              run_eagerly=True)

model.fit(train_ds,
          validation_data=val_ds,
          epochs=5)